# <오늘 할 것: jQuery 부수기>

# 1. page navigation : 게시판 앱 이어서
- 1.1 paging
    - 1.1.1 페이지에 링크 걸기
- 1.2 목록으로 가기
- 1.3 동적 loading

# 2. jQuery
- HTML
- jQuery
    - get 함수

# 3. RequestFactory



# 1. page navigation : 게시판 앱 이어서

## 1.1 paging (page navigation)

paging이 필요한 이유는 한번에 전체 데이터를 모두 전송하는 것이 효율도 떨어질 뿐더러 유저 입장에서도 필요없기 때문이다.

paging을 구현하기 위해서는 django에서 db로 보내는 sql 문장에 **조회하고자 하는 페이지에 대한 정보**를 포함시켜야 한다.

paging은 django의 `Paginator`를 통해서 구현할 수 있다. 먼저 `Paginator`를 조작하는 법을 익히자.

In [ ]:
from django.core.paginator import Paginator

In [ ]:
def list(request):

    datas = [{'id': 1, 'name': '홍길동1'},
            {'id': 2, 'name': '홍길동2'},
            {'id': 3, 'name': '홍길동3'},
            {'id': 4, 'name': '홍길동4'},
            {'id': 5, 'name': '홍길동5'},
            {'id': 6, 'name': '홍길동6'},
            {'id': 7, 'name': '홍길동7'},
            ]

    p = Paginator(datas, 3)   #페이지당 글 수
    sub = p.page(1)         #첫번째 페이지 가져오기

    return render(request, "myboard/page.html", {'datas': sub})

이렇게 했을 때 실제 브라우저에 노출되는 데이터는 
- {'id': 1, 'name': '홍길동1'},
- {'id': 2, 'name': '홍길동2'},
- {'id': 3, 'name': '홍길동3'},

를 템플릿화 한 것이 된다.

## `Paginator`
- `Paginator(전체 데이터, 페이지당 데이터 수)`
    - 전체 데이터: 리스트 형태
    - `.page(호출할 페이지)` 메소드로 분할된 페이지 중 원하는 페이지에 접근할 수 있다.
        - 호출할 페이지 **1부터 시작**하는 정수 인덱스값
            - 이 파라미터를 query 방식으로 url로부터 동적으로 입력받을 수 있도록 다음과 같이 처리할 수 있다.

In [ ]:
def list(request):

    datas = [{'id': 1, 'name': '홍길동1'},
            {'id': 2, 'name': '홍길동2'},
            {'id': 3, 'name': '홍길동3'},
            {'id': 4, 'name': '홍길동4'},
            {'id': 5, 'name': '홍길동5'},
            {'id': 6, 'name': '홍길동6'},
            {'id': 7, 'name': '홍길동7'},
            ]
    page = request.GET.get('page', 1)
    p = Paginator(datas, 3)   #페이지당 글 수
    sub = p.page(page)         #첫번째 페이지 가져오기

    return render(request, "myboard/page.html", {'datas': sub})

이럴 경우 url 요청은 `localhost:8000/myboard/?page=페이지번호` 형식으로 들어와야 한다.

### 1.1.1 페이지에 링크 걸기

템플릿 파일에서 `<a>` 태그를 활용해서 링크를 거는 건 동일하지만, 페이징 기능에 링크를 걸 때는 고려해야할 것들이 몇가지 더 있다. 현재 페이지를 기준으로 링크가 비활성화되어야하는 경우가 몇가지 있기 때문이다.

- 현재 페이지의 링크는 비활성화
- 첫 페이지에 있을 때 `이전으로` 링크 비활성화
- 마지막 페이지에 있을 때 `다음으로` 링크 비활성화

`Paginator` 모듈에는 다양한 기능의 메소드들이 있기 때문에 위의 기능은 해당 메소드들 + 템플릿에서 if문을 반복해서 써서 구현할 수 있다.
- `number`: 현재 페이지 번호
- `has_other_pages`: 다른 페이지가 있는지
- `has_previous`: 이전 페이지가 있는지
- `has_next`: 다음 페이지가 있는지
- `previous_page_number`: 이전 페이지 번호
- `next_page_number`: 다음 페이지 번호
- `paginator.page_range`: 전체 페이지 범위

`myboard`앱에 연동하기 전에 시험해보기 위해 `myboard/page.html`을 새로 만들어준다.

In [ ]:
{% if datas.has_other_pages %}   #다른 페이지가 존재하는지
        {% if datas.has_previous %}   #이전 페이지가 존재하는지
          <a href="?page={{ datas.previous_page_number }}">&laquo;</a>    #이전 페이지 번호
        {% else %}
          <span>&laquo;</span>
        {% endif %}

        {% for i in datas.paginator.page_range %}    #전체 페이지 범위
          {% if datas.number == i %}    #페이지 번호
            <span>{{ i }} </span>
          {% else %}
            <a href="?page={{ i }}">{{ i }}</a>
          {% endif %}
        {% endfor %}
        {% if datas.has_next %}    #다음 페이지가 존재하는지
            <a href="?page={{ datas.next_page_number }}">&raquo;</a>     #다음 페이지 번호
        {% else %}
            <span>&raquo;</span>
        {%endif %}
    {%endif%}

그리고 이걸 기존에 만들어 놓은 게시판 앱의 `myboard/views.py`의 `BoardView` 클래스의 `list` 함수에 연동시킨다.

In [ ]:
        elif mode == 'list':
            username = request.session['username']
            user = User.objects.get(username=username)
            datas = models.Board.objects.all().filter(category=category)
            page = request.GET.get('page', 1)
            p = Paginator(datas, 3)  # 페이지당 글 수
            sub = p.page(page)
            context = {"datas": sub, "category": category, "username": username}
            return render(request, "myboard/list.html", context)

물론 `myboard/list.html`에도 해당 기능을 포함시켜준다.

## 1.2 목록으로 가기

## 1.3 동적 loading

여기서부터는 jQuery를 사용해서 클라이언트 사이드에서 대부분의 동작을 구현할 거다. 루트의 `static` 폴더에 `jquerytest.html`을 새로 만들어서 진행했다. 여기서 사용한 jQuery의 기본적인 문법 사항은 이 문서의 2. jQuery 절에서 다루었으니 되짚어보는 의미에서 그 부분을 먼저 읽고 오면 이해에 도움이 될 것이다.

### 1.3.1 페이지 로딩
`Add` 버튼을 눌렀을 때 페이지 단위로 글이 로딩되도록 하는 코드를 짤 거다. jQuery를 활용해서 비동기방식으로 구현하여 버튼을 누를 때마다 페이지가 리로딩되는 느낌을 없앨 거다.

다음과 같은 플로우로 해당 기능을 구현할 것이다.
> 1. HTML 부분에서
    - `<div>` 태그로 글을 디스플레이할 샘플을 만들고

> 2. jQuery 부분에서
    - 샘플을 `hide` 하고
    - 샘플을 복제해서 제목과 조회수를 바꿔치기 해주고
    - 이걸 디스플레이할 글 수만큼 for 문으로 반복한다.

샘플은 다음과 같이 만든다.

In [ ]:
<div id="view">   #이건 글들이 보일 부분 단위 (board)
    <div id="item">   #글 하나 단위 (post)
        <a href="list?id=4"> <span id="title">제목</span></a>   #바꿔치기할 부분들을 span 태그로 감싼다
        조회수 <span id="cnt"> 5 </span>    #그래서 여기도 span으로 감싼다
        <br>
    </div>
</div>

jQuery 부분은 이렇게 만든다.

In [ ]:
<script>
    $("#item").hide();   #복제용으로 만든 샘플을 숨기고

    var page = 1    #page 초기화

    function getPage(page){
        $.get("/myboard/ajaxget", {page:page}, function(json) {  #서버에서 ajaxget 함수를 호출해서 page 정보를 전달, 리턴을 function에 저장
            console.log(JSON.stringify(json));    #콘솔에 표시
             for (i = 0; i < json.datas.length; i ++) { #datas 길이만큼 for문 실행 (여기서는 3회)
                item = $("#item").clone();  #샘플을 복제해서
                item.show();   #hide까지 복사해버렸으므로 복사 후에는 show 해주고
                 #셀렉터를 찾을 위치를 item으로 한정시켜줘야한다
                $("#title", item).html(json.datas[i].title);  #제목을 바꿔주고
                $("#cnt", item).text(json.datas[i].cnt);     #조회수도 바꿔준다
                $("#view").append(item);    #그리고 view 태그에 바꾼 결과를 더해준다.
            }
        });
    }

    $("#btnAdd").click(function() {   #Add 버튼이 클릭되었을 때 page를 1씩 늘리기 + getPage 함수 실행
        page++;
        getPage(page);
    });
    getPage(page);     #맨 처음 로딩됐을 때 (Add 버튼을 누르지 않아도) page=1 데이터가 로딩되도록

</script>

참고로 `get` 함수가 호출하는 `ajaxget` 함수는 이렇게 생겼다.

In [ ]:
def ajaxget(request):
    page = request.GET.get('page', 1)
    print(request.GET)
    datas = models.Board.objects.all().filter(category = 'common')
    page = int(page)
    sub = datas[(page-1)*3 : page*3]
    datas = {"datas": [{'pk': x.pk, 'title': x.title, 'cnt': x.cnt} for x in sub]}
    return JsonResponse(datas)

이렇게하면 `Add` 버튼을 눌렀을 때 페이지가 리로딩되는 느낌없이 화면에 페이지 단위로 글이 추가된다. 

### 1.3.2 게시글 삭제
이번에는 `지우기` 버튼을 눌렀을 때 글이 삭제되도록 하겠다.

이것도 두 부분으로 나누어서 진행한다.
1. db에서 글 삭제
2. 페이지(클라이언트 뷰)에서 글 삭제


1. db에서 글을 삭제하는 것은 다음과 같은 플로우로 진행되도록 한다.
> `지우기`가 `deleteBoard` 함수 호출(pk 전달) -> `deleteBoard` 함수가 서버에서 `ajaxdel` 함수 호출 -> `ajaxdel` 함수가 db에서 해당 pk의 글 삭제

`지우기`에는 `<a>` 태그의 속성 `href`를 통해서 `deleteBoard` 함수를 호출할 건데, 이때 함수 인자로 전달할 pk가 동적으로 바뀌어야 하기 때문에 문자열이 아닌, **javascript로** `href`를 써줄거다.

In [ ]:
<a id='del' href=javascript:deleteBoard(1)>지우기</a>

얘는 어차피 `hide`처리한 샘플이라서 일단 `deleteBoard` 함수의 인자를 1로 주었고, 이걸 아래에서 다음과 같은 방법으로 동적으로 바뀌도록 처리한다.

In [ ]:
function getPage(page){
    $.get("/myboard/ajaxget", {page:page}, function(json) {
        console.log(JSON.stringify(json));
            for (i = 0; i < json.datas.length; i ++) {
                item = $("#item").clone();
                item.show();
                $("#title", item).html(json.datas[i].title);
                $("#del", item).attr("href","javascript:deleteBoard(" + json.datas[i].pk + ")");  #여기다!
                $("#cnt", item).text(json.datas[i].cnt);
                $("#view").append(item);
            }
        });
    }

html 태그의 속성을 바꾸고 싶을 때는 `attrs` 메소드를 쓰면 된다. (페이지에 실제로 보이는, 태그로 감싸진 부분을 수정하고 싶으면 `html` 메소드를 쓴다)

`$(선택할 태그).attrs(속성명, 새로운 속성값)`

이렇게 하면 글 별 `지우기`에 제대로된 링크를 걸어주었다. 이제 `deleteBoard` 함수를 정의할 차례다. 클라이언트 사이드에서 처리할 것이므로 `<script> </script>` 부분에 작성해준다.

In [ ]:
function deleteBoard(pk){
    $.get("/myboard/ajaxdel", {pk:pk}, function(json) {
        alert(json.result);
    });
};

이 함수가 실행될 때 벌어지는 일은
- `get`함수가 서버에서 `pk`를 파라미터로 전달하면서 `ajaxdel` 함수를 호출하고
- (여기서 `get` 함수는 종료되고 `ajaxdel` 함수는 별도의 thread로 실행된다)
- 함수 리턴값을 `json`이라는 변수에 저장해서
- `json` 변수의 `result`라는 키에 해당하는 값을 브라우저에 알림창으로 띄우는 것이다.

`myboard/views.py`에 정의된 `ajaxdel` 함수는 이렇게 생겼다.

In [ ]:
def ajaxdel(request):
    pk = request.GET.get("pk")
    board = models.Board.objects.get(pk=pk)
    title = board.title
    board.delete()
    result = f'{title}을 성공적으로 삭제했습니다.'
    return JsonResponse({'error': 0, 'result': result})

이 함수의 리턴값에서 'result'를 alert 창에 띄운다는 것은 곧 `지우기`를 클릭했을 때 `{어떤글의 제목}을 성공적으로 삭제했습니다.`가 alert 창에 뜬다는 뜻이다.

`ajaxdel`함수를 실행함으로써 1. db에서 글 삭제는 성공했으니, 이제는 2. 페이지(클라이언트 뷰)에서 글 삭제를 해야한다. 

1.3.1 페이지 로딩과 마찬가지로 비동기 방식으로 삭제 후에 리프레시를 하지 않아도 브라우저에서 사라지도록 처리할 거다. 그런데 **삭제할 노드를 어떻게 특정하여 코드화할 것인지**의 문제가 있다. 이를 해결하기 위해서

### 노드의 attribute 중 id를 노드별 고유값으로 재지정한다.



In [ ]:
function getPage(page){
    $.get("/myboard/ajaxget", {page:page}, function(json) {
        console.log(JSON.stringify(json));
            for (i = 0; i < json.datas.length; i ++) {
                item = $("#item").clone();
                item.attr("id", "item_" + json.datas[i].pk)      #여기다!
                item.show();
                $("#title", item).html(json.datas[i].title);
                $("#del", item).attr("href","javascript:deleteBoard(" + json.datas[i].pk + ")");  
                $("#cnt", item).text(json.datas[i].cnt);
                $("#view").append(item);
            }
        });
    }

삭제를 위해 아이템별로 고유 id를 부여한다. 브라우저에 띄워서 소스를 확인해보면 노드별로 id가 잘 재지정된 것을 확인할 수 있다.

<img src=item_id.jpg>

이제 이 고유 id를 활용해서 원하는 노드를 페이지에서 삭제할 수 있다.


jQuery에서 `append`의 반대는 `remove`이다. 아까 정의했던 `deleteBoard` 함수에 한 줄만 더 추가해주면 된다.


In [ ]:
function deleteBoard(pk){
    $.get("/myboard/ajaxdel", {pk:pk}, function(json) {
    alert(json.result);
    $("#item_" + pk).remove();  #여기를 추가!
    });
};

실행해보면 db에서도, 페이지에서도 성공적으로 잘 삭제되는 것을 확인할 수 있다.

# 2. jQuery

> jQuery(제이쿼리)는 HTML의 클라이언트 사이드 조작을 단순화 하도록 설계된 크로스 플랫폼의 **자바스크립트 라이브러리**다. 오늘날 가장 인기있는 자바스크립트 라이브러리 중 하나이다.

출처: 위키피디아

## HTML
- `<div> </div>` 하나의 논리적인 그룹으로 묶기

- `<span> </span>` 하나의 요소를 집어내기

- `console.log(데이터)` 콘솔에서 확인하기
    - ctrl + shift + i로 브라우저에서 콘솔을 열 수 있다.
    
## jQuery
- HTML 부분의 태그를 jQuery에서 선택하려면...
    - `<tag>` --> `$("tag")`
    - `<tag id=아이디>` --> `$("#아이디")`
    - `<tag class=클래스>` --> `$(".클래스")`

###  $.get(url, param, function(var) { 실행내용 } )
- url: str; 절대경로(base 디렉토리 뒤에 붙는다)
- param: json; url 뒤에 query 형태로 붙을 파라미터들
- function: 예약어; 바꾸면 안됨
- var: 함수의 실행 결과를 받을 변수명
- 실행내용: var을 가지고 뭔가를 하기
    
#### get 함수는 http 요청을 시작시키기만 하는 함수다.
- 일단 http thread를 시작시키고 나면 get 함수는 종료된다.
- http 요청은 별개의 thread로 돌고 -> 데이터를 받아서 -> 지정해준 작업을 수행한다.

# 3. RequestFactory

> 서버를 띄우지 않아도 작성한 django 앱을 코드 by 코드로 테스트해볼 수 있는 짱 좋은 라이브러리

가상의 url 및 request 메세지도 생성할 수 있고, 각종 생성자, 함수, 메소드 등의 리턴값도 확인할 수 있다.

django-shell에서 다음과 같이 import 해서 쓸 수 있다.

In [9]:
from django.test import RequestFactory
rf = RequestFactory()   #리퀘스트팩토리 객체를 생성해서 쓴다.

예를 들어 `Paginator`를 활용해서 작성한 코드를 테스트해보고 싶다고 하자.

In [16]:
from django.shortcuts import render, get_object_or_404, redirect
from django.core.paginator import Paginator

In [19]:
url = '/myboard/ajaxget?page=2'   #존재하지 않는 url이어도 된다.
request = rf.get(url)    #가상의 request 메시지를 만들어준다.

page = request.GET.get('page', 1)
print(page)

2


In [21]:
datas = [{'id': 1, 'name': '홍길동1'},
         {'id': 2, 'name': '홍길동2'},
         {'id': 3, 'name': '홍길동3'},
         {'id': 4, 'name': '홍길동4'},
         {'id': 5, 'name': '홍길동5'},
         {'id': 6, 'name': '홍길동6'},
         {'id': 7, 'name': '홍길동7'},
        ]

p = Paginator(datas, 3)
sub = p.page(page)
response = render(request, "myboard/page.html", {'datas': sub})

In [22]:
response

<HttpResponse status_code=200, "text/html; charset=utf-8">

In [23]:
print(response.content.decode())





   4  / 홍길동4  <br>



   5  / 홍길동5  <br>



   6  / 홍길동6  <br>




        
          <a href="?page=1">&laquo;</a>
        

        
          
            <a href="?page=1">1</a>
          
        
          
            <span>2 </span>
          
        
          
            <a href="?page=3">3</a>
          
        
        
            <a href="?page=3">&raquo;</a>
        
    

